In [124]:
# Task 3: Machine Learning Modeling for Insurance Premium and Claims
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

In [125]:
# Load cleaned dataset
df = pd.read_csv(r"C:\Users\hp\Desktop\10 Acadamy\VS code\insurance-risk-analytics\insurance-risk-analytics-week3\data\insurance_data_eda.csv")

C:\Users\hp\AppData\Local\Temp\ipykernel_3384\2291897501.py:2: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\hp\Desktop\10 Acadamy\VS code\insurance-risk-analytics\insurance-risk-analytics-week3\data\insurance_data_eda.csv")


In [126]:
# Clean numeric string columns (replace commas with dots, then convert to float)
df['CustomValueEstimate'] = df['CustomValueEstimate'].apply(lambda x: str(x).replace(',', '.')).astype(float)

In [127]:
# Define features and targets
X = df.drop(columns=["TotalPremium", "TotalClaims"])
y_premium = df["TotalPremium"]
y_claims = df["TotalClaims"]

In [128]:
# Train/test split
X_train, X_test, y_train_p, y_test_p = train_test_split(X, y_premium, test_size=0.2, random_state=42)
_, _, y_train_c, y_test_c = train_test_split(X, y_claims, test_size=0.2, random_state=42)

In [129]:
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)

In [ ]:
y_train_p = y_train_p.loc[X_train.index]
y_test_p = y_test_p.loc[X_test.index]

In [131]:
# Identify column types
numeric_features = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X_train.select_dtypes(include=["object", "bool", "category"]).columns.tolist()

In [132]:
# Identify numeric and categorical features
numeric_features = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X_train.select_dtypes(include=["object", "bool", "category"]).columns.tolist()

In [133]:
# Convert object-type numeric strings to float
for col in categorical_features:
    if col in X_train.columns:
        try:
            X_train[col] = X_train[col].astype(str).str.replace(',', '.').astype(float)
            X_test[col] = X_test[col].astype(str).str.replace(',', '.').astype(float)
        except:
            pass  # Skip if not convertible

In [134]:
# Re-identify column types after conversion
numeric_features = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X_train.select_dtypes(include=["object", "bool", "category"]).columns.tolist()

In [135]:
# Remove IDs and transaction month from processing
exclude_cols = ['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth']
numeric_features = [col for col in numeric_features if col not in exclude_cols]
categorical_features = [col for col in categorical_features if col not in exclude_cols]

In [136]:
# Re-identify columns after conversion
numeric_features = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X_train.select_dtypes(include=["object", "bool", "category"]).columns.tolist()

In [ ]:
numeric_features = [
    'PostalCode', 'mmcode', 'RegistrationYear', 'Cylinders', 'cubiccapacity',
    'kilowatts', 'NumberOfDoors', 'CustomValueEstimate', 'CapitalOutstanding',
    'NumberOfVehiclesInFleet', 'SumInsured', 'CalculatedPremiumPerTerm',
    'ClaimMade', 'LossRatio'
]


In [ ]:
categorical_features = [
    'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language', 'Bank',
    'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
    'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'VehicleType', 'make',
    'Model', 'bodytype', 'AlarmImmobiliser', 'TrackingDevice', 'NewVehicle',
    'WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 'TermFrequency',
    'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
    'Product', 'StatutoryClass', 'StatutoryRiskType'
]

In [137]:
# Preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [138]:
# Column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [139]:
# Linear Regression Pipeline
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [140]:
# Fit and predict
lr_pipeline.fit(X_train, y_train_p)
y_pred_lr = lr_pipeline.predict(X_test)

ValueError: Found array with 0 sample(s) (shape=(0, 52)) while a minimum of 1 is required by SimpleImputer.

In [ ]:
# Random Forest Pipeline
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Fit and predict
rf_pipeline.fit(X_train, y_train_p)
y_pred_rf = rf_pipeline.predict(X_test)

In [ ]:
# Evaluation function
def evaluate_model(y_test, y_pred, model_name):
    print(f"----- {model_name} -----")
    print("MAE :", mean_absolute_error(y_test, y_pred))
    print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
    print("R^2 :", r2_score(y_test, y_pred))
    print()

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [ ]:
# Evaluate models
evaluate_model(y_test_p, y_pred_lr, "Linear Regression (Premium)")
evaluate_model(y_test_p, y_pred_rf, "Random Forest (Premium)")

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'No excess'